In [1]:
# Imports
import pandas as pd
import os
import locale
from matplotlib import pyplot as plt
import numpy as np
from prophet import Prophet

locale.setlocale(locale.LC_ALL, 'fr_FR');

ModuleNotFoundError: No module named 'prophet'

In [ ]:
# Chemins
path = None
path = os.path.dirname(os.getcwd())
print("Le chemin est "+path)

pdf_dir = os.path.join(path, 'data', 'data_cine_pdf')
raw_data_path = os.path.join(path, 'data', 'raw_data', 'df_raw.csv')
data_path = os.path.join(path, 'data', 'clean_data.csv')

In [ ]:
# Recuperation des données
df = pd.read_csv(data_path, parse_dates=[['date', 'time']], dayfirst=True)

In [ ]:
df.info();

In [ ]:
df

### Description des données

- **Film** : Titre du film (chaîne de caractères)
- **Version** : Langue des dialogues ('_FR_' pour film français en langue originale, '_VF_' pour film étranger en langue française et '_VO_' pour film étranger en langue originale)
- **Relief** : Option supplémentaire ('_2D_' si l'option est désactivée, '_3D_' si l'option est activée)
- **Payants** : Nombre d'entrées vendues (valeur numérique)
- **Gratuits** : Nombre d'entrées gratuites (valeur numérique)
- **Places libres** : Nombre de places restantes dans la salle (valeur numérique)
- **Taux remplissage** : Niveau de remplissage de la salle (pourcentage, calculé par : (nombre d'entrées payantes + nombre d'entrées gratuites) / nombre de places)

### Nettoyage et preparation

In [ ]:
# Creation de la nouvelle colonne 'Nombre entrees'
df['Nombre entrees'] = df[["Payants", "Gratuits"]].sum(axis=1)

In [ ]:
# Nettoyage des nombres d'entrées à 0
# df = df[~(df['Nombre entrees'] == 0)].reset_index()

In [ ]:
# Conversion en valeur numérique de la colonne 'Taux remplissage'
try:
    df['Taux remplissage'] = df['Taux remplissage'].str.rstrip('%').str.replace(',','.').astype('float')
except:
    print('Nothing to do')

In [ ]:
df

### Statistiques descriptives

In [ ]:
# Dates
print("Le jeu de données commence le "+df['date_time'].min().strftime('%d %B %Y') + " et finit le "+df['date_time'].max().strftime('%d %B %Y'))

# Nombre de films
print(str(len(df['Film'].unique()))+" films ont été proposés")

In [ ]:
# Version
plt.subplot(1, 2, 1)
df['Version'].value_counts().plot(kind='bar',
                                  xlabel='Version');
plt.title('Nombre de séances en fonction de la version du film', fontsize=16);

plt.subplot(1, 2, 2)
df.groupby('Version').sum()['Nombre entrees'].plot(kind='bar',
                                                   figsize=(20, 6));
plt.title('Nombre d\'entrées en fonction de la version du film', fontsize=16);

In [ ]:
# Relief
plt.subplot(1, 2, 1)
df['Relief'].value_counts().plot(kind='bar',
                                  xlabel='Relief');
plt.title('Nombre de séances en fonction du relief', fontsize=16);

plt.subplot(1, 2, 2)
df.groupby('Relief').sum()['Nombre entrees'].plot(kind='bar',
                                                   figsize=(20, 6));
plt.title('Nombre d\'entrées en fonction du relief', fontsize=16);

In [ ]:
# Taux remplissage
print("Le taux de remplissage moyen est de "+str(round(df['Taux remplissage'].mean(),2))+" %")

In [ ]:
# Nombre d'entrées par année
df.groupby(df['date_time'].dt.year.astype(str)).sum()['Nombre entrees'].plot(kind='bar',
                                                                             figsize=(20, 6),
                                                                             xlabel='Année');
plt.title('Nombre d\'entrées par année', fontsize=16);

In [ ]:
# Moyennes par salle
df.groupby('Salle').mean()

In [ ]:
# Moyennes par année
df.groupby(df['date_time'].dt.year).mean()

In [ ]:
# Moyennes par année
plt.figure(figsize=(20, 8))
plt.plot(df.groupby(df['date_time'].dt.year.astype(str)).mean()['Payants'], label='Payants', color='green')
plt.plot(df.groupby(df['date_time'].dt.year.astype(str)).mean()['Nombre entrees'], label='Total', color='blue')

plt.legend(title='Entrées')

plt.ylabel('Nombre d\'entrées moyen', fontsize=14)
plt.xlabel('Années', fontsize=14)
plt.title('Nombre d\'entrées moyen par année', fontsize=16)


plt.show()

In [ ]:
# Taux de remplissage par salle et par année
df.groupby(['Salle', df['date_time'].dt.year.astype(str)]).mean()['Taux remplissage'].unstack(level=0).plot(figsize=(20,8));

plt.ylabel('Taux de remplissage (%)', fontsize=14)
plt.xlabel('Années', fontsize=14)
plt.title('Taux de remplissage par salle et par année', fontsize=16)


plt.show();

In [ ]:
# Top nombre d'entrées par film
df.groupby('Film').sum()['Nombre entrees'].sort_values(ascending=True).tail(10).plot(kind='barh',figsize=(20,13));

plt.ylabel('Nombre total d\'entrées', fontsize=14)
plt.xlabel('Film', fontsize=14)
plt.title('Nombre total d\'entrees par film (Top 10)', fontsize=16)


plt.show();

### Analyse de la série temporelle

In [ ]:
# Préparation données pour Prophet
df_prophet = pd.DataFrame()
df_prophet['ds'] = df["date_time"]
df_prophet['y'] = df['Nombre entrees']

In [ ]:
# aggrégation sur le time stamp
df_prophet= df_prophet.groupby(['ds']).sum()
df_prophet['ds'] = df_prophet.index
df_prophet = df_prophet.reindex(columns = ['ds', 'y'])

In [ ]:
# réechantillonnage à la journée
r = df_prophet.resample('1d')
df_day = r.agg(np.sum)
df_day['ds'] = df_day.index
df_day = df_day.reindex(columns = ['ds', 'y'])
df_day.plot('ds','y', figsize = (15,10))
plt.show()